In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)
    return images, labels

In [3]:
def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x,y

In [4]:
x_train, y_train = load_mnist('',kind='train')
x_test, y_test = load_mnist('',kind='t10k')

In [5]:
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0
x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

In [6]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=x_train.shape),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [8]:
model = Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
375/375 [==============================] - 2s 4ms/step - loss: 1.5483 - accuracy: 0.9186
Epoch 2/10
375/375 [==============================] - 1s 2ms/step - loss: 1.5261 - accuracy: 0.9351
Epoch 3/10
375/375 [==============================] - 1s 2ms/step - loss: 1.5206 - accuracy: 0.9415
Epoch 4/10
375/375 [==============================] - 1s 2ms/step - loss: 1.5173 - accuracy: 0.9445
Epoch 5/10
375/375 [==============================] - 1s 2ms/step - loss: 1.5152 - accuracy: 0.9472
Epoch 6/10
375/375 [==============================] - ETA: 0s - loss: 1.5161 - accuracy: 0.94 - 1s 2ms/step - loss: 1.5162 - accuracy: 0.9459
Epoch 7/10
375/375 [==============================] - 1s 2ms/step - loss: 1.5134 - accuracy: 0.9477
Epoch 8/10
375/375 [==============================] - 1s 2ms/step - loss: 1.5127 - accuracy: 0.9486
Epoch 9/10
375/375 [==============================] - 1s 2ms/step - loss: 1.5088 - accuracy: 0.9529
Epoch 10/10
375/375 [==============================] - 1s 

In [12]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

63/63 - 0s - loss: 1.5252 - accuracy: 0.9350

Test accuracy: 0.9350000023841858
